In [1]:
from controller.blockchain import Blockchain
blockchain = Blockchain(file_path="Proof_of_ownership.docx", chunks=10)

In [2]:
splitted_data = blockchain._chunk_data()

In [3]:
for index, bin_data in enumerate(splitted_data):
    data = blockchain.encrypt_block(bin_data)
    previous_block = blockchain.get_previous_block()
    previous_nonce = previous_block['nonce']
    nonce, current_hash = blockchain.nonce_and_hash(previous_nonce)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(data, nonce, current_hash, previous_hash, title=blockchain.file)

In [4]:
from pprint import pprint
pprint(blockchain.chain)

[{'block_title': 'Proof_of_ownership.docx_0001',
  'data': 'Blockchain created by: Geoffroy Givry',
  'hash': '00000000000000000',
  'index': 1,
  'nonce': 1,
  'previous_hash': '0',
  'timestamp': '2018-09-20T10:20:48.256963'},
 {'block_title': 'Proof_of_ownership.docx_0002',
  'data': {'ciphertext': '...',
   'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
   'nonce': '5X6CZRgY5NdDQ+5QM9AXNQ==',
   'tag': 'T0xWse6zp13nMpTAanL+Hw=='},
  'hash': '0000c00870f23a23ae80377298491b091db400d575be0efbde5b310f2f763ed1',
  'index': 2,
  'nonce': 533,
  'previous_hash': 'a99b222103432f52d4a62e26a5bea48938ad84ac598760549b4126ddeace6806',
  'timestamp': '2018-09-20T10:20:48.306774'},
 {'block_title': 'Proof_of_ownership.docx_0003',
  'data': {'ciphertext': '...',
   'header': 'UHJvb2Zfb2Zfb3duZXJzaGlwLmRvY3ggc2VudCBieSBHZW9mZnJveSBHaXZyeS4=',
   'nonce': 'GtGwrNU29Uas5EeMCSlh+Q==',
   'tag': 'AobrGIx17GILawsO0heCNw=='},
  'hash': '0000b8318b7ca4e2bc9be063d7dfcb0d0e70f

In [5]:
blockchain.is_chain_valid(blockchain.chain)

True

In [6]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

def generate_keys(private_key_file="private.pem", public_key_file="receiver.pem"):
    key = RSA.generate(2048)

    private_key = key.export_key()
    public_key = key.publickey().export_key()

    with open(private_key_file, "wb") as file_out:
        file_out.write(private_key)

    with open(public_key_file, "wb") as file_out:
        file_out.write(public_key)
    return private_key, public_key

private_key, public_key = generate_keys()



def encrypt_data(data, file_out, public_key_file="receiver.pem"):
    if isinstance(data, bytes):
        data = data
    else:
        data = data.encode("utf-8")

    with open(file_out, "wb") as file_out:

        recipient_key = RSA.import_key(open("receiver.pem").read())
        session_key = get_random_bytes(16)

        # Encrypt the session key with the public RSA key
        cipher_rsa = PKCS1_OAEP.new(recipient_key)
        enc_session_key = cipher_rsa.encrypt(session_key)

        # Encrypt the data with the AES session key
        cipher_aes = AES.new(session_key, AES.MODE_EAX)
        ciphertext, tag = cipher_aes.encrypt_and_digest(data)
        [ file_out.write(x) for x in (enc_session_key, cipher_aes.nonce, tag, ciphertext) ]

data = get_random_bytes(16)
file_out = "encrypted_data.bin"

generate_keys()
encrypt_data(data, file_out)

In [7]:
def decrypt_data(file_in, private_key_file="private.pem"):
    with open(file_in, "rb") as file_in:
        
        private_key = RSA.import_key(open("private.pem").read())

        enc_session_key, nonce, tag, ciphertext = \
           [ file_in.read(x) for x in (private_key.size_in_bytes(), 16, 16, -1) ]

        # Decrypt the session key with the private RSA key
        cipher_rsa = PKCS1_OAEP.new(private_key)
        session_key = cipher_rsa.decrypt(enc_session_key)

        # Decrypt the data with the AES session key
        cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
        data = cipher_aes.decrypt_and_verify(ciphertext, tag)
        return data

file_in = "encrypted_data.bin"
my_message = decrypt_data(file_in)
print(my_message)

b'\x97\x8e\xc0\x109\xd9\x88\x81\xe7\xb9\xa9\x19\xee\xed\x8e\xe6'
